In [1]:
print('intelligibility eval')

intelligibility eval


In [ ]:
!pip install transformers
!pip install torch
!pip install jiwer
!pip install pandas==1.1.5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 3.5 MB/s eta 0:00:0000:0100:01m
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3


In [3]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "nilc-nlp/distil-whisper-coraa-mupe-asr"
#model_id = "sidleal/distil-whisper-tarsila-asr-v1-200k"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    #model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
    model_id, torch_dtype=torch_dtype, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    torch_dtype=torch_dtype,
    return_timestamps=True,
    device=device,
)

def trancript_with_distil(file_path):
    result = pipe(file_path)
    return result["text"]

Device set to use cuda:0


In [4]:
result = trancript_with_distil("/home/sidleal/sources/FastSpeech2/output/result/cmltts_entoa_prossegue/sp234/a gente ia ao cinema tinha em são paulo tem uns cinemas ótimos.wav")
print("----------------------------------------")
print(result)
print("----------------------------------------")

/home/sidleal/anaconda3/envs/ipynb12/lib/python3.12/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


----------------------------------------
mas a gente que ia assim que você tinha que ser bar com os seus vejinhos
----------------------------------------


In [5]:
import re
import jiwer
import jiwer.transforms as tr

cer_transform = tr.Compose(
    [
        jiwer.ToLowerCase(),
        jiwer.RemoveMultipleSpaces(),
        jiwer.Strip(),
        jiwer.ReduceToListOfListOfChars(),
    ]
)

# It's the jiwer default transform
wer_transform = jiwer.Compose([
    jiwer.ToLowerCase(),
    jiwer.RemoveMultipleSpaces(),
    jiwer.Strip(),
    jiwer.ReduceToListOfListOfWords(),
])

def compute_cer(reference, hypothesis):
    reference = reference.lower()
    hypothesis = hypothesis.lower()
    cer = jiwer.wer(reference, hypothesis, truth_transform=cer_transform, hypothesis_transform=cer_transform)
    return cer

def compute_wer(reference, hypothesis):
    reference = reference.lower()
    hypothesis = hypothesis.lower()
    wer = jiwer.wer(reference, hypothesis, truth_transform=wer_transform, hypothesis_transform=wer_transform)
    return wer


alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZÇÃÀÁÂÊÉÍÓÔÕÚÛabcdefghijklmnopqrstuvwxyzçãàáâêéíóôõũúû1234567890%\\-\n/\\ "


def replace_special_tokens_and_normalize(text):
    text = text.lower()

    map_words = {
        "éh": "eh",
        "ehm": "eh",
        "ehn": "eh",
        "hum": "uh",
        "hm": "uh",
        "uhm": "uh",
        "hã": "ah",
        "ãh": "ah",
        "ã":  "ah",
        "hmm": "uh",
        "mm": "uh",
        "mhm": "uh"
    }

    text = re.sub("h+", "h", text)
    text = re.sub("[^{}]".format(alphabet+" "), " ", text)
    text = re.sub("[ ]+", " ", text)

    words = text.split(' ')
    new_words = []
    for word in words:
        if word == '' or word == ' ':
            continue
        if word in map_words:
            new_words.append(map_words[word])
        else:
            new_words.append(word)

    return " ".join(new_words)


def calculate_wer_cer(reference, hypothesis):
    if reference.strip() == '' or hypothesis.strip() == '':
        return 1, 1
    wer = compute_wer(reference, hypothesis)
    cer = compute_cer(reference, hypothesis)
    return wer, cer



In [8]:
#sintetizar
import pandas as pd
import subprocess

file_path = "/home/sidleal/sources/entoa-tts/experiments/sp_234_l1_transc_cer_wer.csv"
df = pd.read_csv(file_path)
df.head()


,path,human,ground,auto,prosodic,ground_wer,ground_cer,auto_wer,auto_cer,prosodic_wer,prosodic_cer
0,SP_DID_234_seg_229.22_253.82.wav,bom um bom artista é o que desempenha o papel ...,bom um bom artista é o que desempenha o papel ...,tão bom artista é o que desempenha o papel na ...,dom um bom artista,0.0179,0.0148,0.7500,0.5535,0.9464,0.9336
1,SP_DID_234_seg_305.70_309.20.wav,a inês que tem assistido várias peças mas eu n...,aí nesse que tenha sido várias peças mas eu nã...,aí que tem que levar a visita da casa assim só...,as buro e tem tem que tem as mas essas essas p...,0.2353,0.1358,0.8824,0.6790,0.9412,0.6790
2,SP_DID_234_seg_413.92_427.90.wav,sei lá eu sabe o que eu a o que eu noto que o ...,sei lá eu sabe que eu que eu noto que o teatro...,se lá eu saber o que o homem é motor eu peguei...,se lá eu sabia que geovrô não tinha que ver né...,0.1429,0.0818,0.7959,0.6909,0.8367,0.6773
3,SP_DID_234_seg_1146.79_1147.03.wav,fala,tando,fava,fala,1.0000,1.0000,1.0000,0.2500,0.0000,0.0000
4,SP_DID_234_seg_1207.41_1208.51.wav,ai que horror,ai que horror,aí que eu vou,ah em que horror,0.0000,0.0000,1.0000,0.4615,0.6667,0.3077


In [ ]:
source_path = "/home/sidleal/sources/FastSpeech2/output/result/cmltts_entoa_prossegue/sp234/"

for index, row in df.iterrows():    
    human_value = row['human']
    file_name = human_value + ".wav"
    if len(human_value) > 100:
        file_name = human_value[:100] + ".wav"
    print(f"Row {index}: {file_name} - {human_value}")
    result = trancript_with_distil(source_path + "/" + file_name)
    print("-->", result)
    df.at[index, 'prossegue'] = result


In [10]:
df.head()

,path,human,ground,auto,prosodic,ground_wer,ground_cer,auto_wer,auto_cer,prosodic_wer,prosodic_cer,prossegue
0,SP_DID_234_seg_229.22_253.82.wav,bom um bom artista é o que desempenha o papel ...,bom um bom artista é o que desempenha o papel ...,tão bom artista é o que desempenha o papel na ...,dom um bom artista,0.0179,0.0148,0.7500,0.5535,0.9464,0.9336,bombo artista é o que desempenhou até uma peça...
1,SP_DID_234_seg_305.70_309.20.wav,a inês que tem assistido várias peças mas eu n...,aí nesse que tenha sido várias peças mas eu nã...,aí que tem que levar a visita da casa assim só...,as buro e tem tem que tem as mas essas essas p...,0.2353,0.1358,0.8824,0.6790,0.9412,0.6790,a inesquec que tinha assistido várias peças ma...
2,SP_DID_234_seg_413.92_427.90.wav,sei lá eu sabe o que eu a o que eu noto que o ...,sei lá eu sabe que eu que eu noto que o teatro...,se lá eu saber o que o homem é motor eu peguei...,se lá eu sabia que geovrô não tinha que ver né...,0.1429,0.0818,0.7959,0.6909,0.8367,0.6773,se não saber o que não ou eu moro do meu tempo...
3,SP_DID_234_seg_1146.79_1147.03.wav,fala,tando,fava,fala,1.0000,1.0000,1.0000,0.2500,0.0000,0.0000,fala pausa pausa pausa pausa pausa pausa pausa...
4,SP_DID_234_seg_1207.41_1208.51.wav,ai que horror,ai que horror,aí que eu vou,ah em que horror,0.0000,0.0000,1.0000,0.4615,0.6667,0.3077,aí que eu vou é que eu vou


In [11]:
for index, row in df.iterrows():    
    human_value = row['human']
    prossegue_value = row['prossegue']
    human_norm = replace_special_tokens_and_normalize(human_value)
    prossegue_norm = replace_special_tokens_and_normalize(prossegue_value)
    wer, cer = calculate_wer_cer(human_norm, prossegue_norm)
    df.at[index, 'prossegue_wer'] = wer
    df.at[index, 'prossegue_cer'] = cer

In [12]:
df.head()

,path,human,ground,auto,prosodic,ground_wer,ground_cer,auto_wer,auto_cer,prosodic_wer,prosodic_cer,prossegue,prossegue_wer,prossegue_cer
0,SP_DID_234_seg_229.22_253.82.wav,bom um bom artista é o que desempenha o papel ...,bom um bom artista é o que desempenha o papel ...,tão bom artista é o que desempenha o papel na ...,dom um bom artista,0.0179,0.0148,0.7500,0.5535,0.9464,0.9336,bombo artista é o que desempenhou até uma peça...,0.875000,0.708487
1,SP_DID_234_seg_305.70_309.20.wav,a inês que tem assistido várias peças mas eu n...,aí nesse que tenha sido várias peças mas eu nã...,aí que tem que levar a visita da casa assim só...,as buro e tem tem que tem as mas essas essas p...,0.2353,0.1358,0.8824,0.6790,0.9412,0.6790,a inesquec que tinha assistido várias peças ma...,0.647059,0.419753
2,SP_DID_234_seg_413.92_427.90.wav,sei lá eu sabe o que eu a o que eu noto que o ...,sei lá eu sabe que eu que eu noto que o teatro...,se lá eu saber o que o homem é motor eu peguei...,se lá eu sabia que geovrô não tinha que ver né...,0.1429,0.0818,0.7959,0.6909,0.8367,0.6773,se não saber o que não ou eu moro do meu tempo...,0.877551,0.712329
3,SP_DID_234_seg_1146.79_1147.03.wav,fala,tando,fava,fala,1.0000,1.0000,1.0000,0.2500,0.0000,0.0000,fala pausa pausa pausa pausa pausa pausa pausa...,8.000000,12.000000
4,SP_DID_234_seg_1207.41_1208.51.wav,ai que horror,ai que horror,aí que eu vou,ah em que horror,0.0000,0.0000,1.0000,0.4615,0.6667,0.3077,aí que eu vou é que eu vou,2.333333,1.384615


In [13]:
df.to_csv('sp_234_l1_transc_wer_cer_prossegue.csv', index=False)

In [15]:
from scipy import stats
#import pandas as pd
#df = pd.read_csv('sp_234_l1_transc_wer_cer_prossegue.csv')

t_statistic, p_value = stats.ttest_ind(df["auto_wer"], df["prossegue_wer"])

print(f"WER WHISPER: {df["auto_wer"].mean()*100:.2f}")
print(f"WER PROSSEGUE: {df["prossegue_wer"].mean()*100:.2f}")

print(f"T-statistic: {t_statistic:.4f} ({t_statistic})", f"P-value: {p_value:.4f} ({p_value})" )

t_statistic, p_value = stats.ttest_ind(df["prosodic_wer"], df["prossegue_wer"])

print(f"WER MANUAL: {df["prosodic_wer"].mean()*100:.2f}")
print(f"WER PROSSEGUE: {df["prossegue_wer"].mean()*100:.2f}")

print(f"T-statistic: {t_statistic:.4f} ({t_statistic})", f"P-value: {p_value:.4f} ({p_value})" )

t_statistic, p_value = stats.ttest_ind(df["ground_wer"], df["prossegue_wer"])

print(f"WER HUMAN: {df["ground_wer"].mean()*100:.2f}")
print(f"WER PROSSEGUE: {df["prossegue_wer"].mean()*100:.2f}")

print(f"T-statistic: {t_statistic:.4f} ({t_statistic})", f"P-value: {p_value:.4f} ({p_value})" )

WER WHISPER: 49.60
WER PROSSEGUE: 76.71
T-statistic: -4.7670 (-4.76701094136319) P-value: 0.0000 (2.263081647713262e-06)
WER MANUAL: 43.18
WER PROSSEGUE: 76.71
T-statistic: -5.8896 (-5.889587761951122) P-value: 0.0000 (5.940187933927828e-09)
WER HUMAN: 16.06
WER PROSSEGUE: 76.71
T-statistic: -10.9522 (-10.95220806135418) P-value: 0.0000 (6.317010334490647e-26)


# old code

In [ ]:
import os
import csv

file_name = 'prossegue_cer_wer_output.csv'

folder_path = '/home/sidleal/sources/FastSpeech2/output/result/cmltts_entoa_prossegue/sp_234'

with open(file_name, 'a', newline='') as csv_file:
    writer = csv.writer(csv_file)

    csv_data = ["audio_name", "human_text", "human_text_norm", "transcript_original", "transcript_synthesized", "cer_original", "cer_synthesized", "wer_original", "wer_synthesized"]
    writer.writerow(csv_data)

    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            audio_name = filename[:-8]
            original_audio_name = filename[:-4]
            synth_audio_name = filename[:-4] + "_synthesized.wav"

            with open(f"{folder_path}/{filename}", encoding='utf-8') as f:
                text = f.read()
                text_norm = replace_special_tokens_and_normalize(text)

                transcript_ori = trancript_with_distil(f"{folder_path}/{original_audio_name}")
                transcript_syn = trancript_with_distil(f"{folder_path}/{synth_audio_name}")

                transcript_ori_norm = replace_special_tokens_and_normalize(transcript_ori)
                transcript_syn_norm = replace_special_tokens_and_normalize(transcript_syn)

                ori_wer_cer = calculate_wer_cer(text_norm, transcript_ori_norm)
                syn_wer_cer = calculate_wer_cer(text_norm, transcript_syn_norm)

                print("--------------------------------")
                print(audio_name)
                print("-", text)
                print("-", text_norm)
                print("-", original_audio_name)
                print("-", synth_audio_name)
                print("-", transcript_ori_norm)
                print("-", transcript_syn_norm)
                print(ori_wer_cer)
                print(syn_wer_cer)

                csv_data = [audio_name, text, text_norm, transcript_ori_norm, transcript_syn_norm, ori_wer_cer[1], syn_wer_cer[1], ori_wer_cer[0], syn_wer_cer[0]]
                writer.writerow(csv_data)

            #break



In [ ]:
from scipy import stats
import pandas as pd

df = pd.read_csv(file_name)

t_statistic, p_value = stats.ttest_ind(df["wer_original"], df["wer_synthesized"])

print(f"WER ORIGINAL: {df["wer_original"].mean()*100:.2f}")
print(f"WER SINTETIZADO: {df["wer_synthesized"].mean()*100:.2f}")

print(f"T-statistic: {t_statistic:.4f} ({t_statistic})")
print(f"P-value: {p_value:.4f} ({p_value})" )

WER ORIGINAL: 13.70
WER SINTETIZADO: 23.86
T-statistic: -6.1082 (-6.108212784547283)
P-value: 0.0000 (5.242308079569724e-09)


# f5tts

In [1]:
import pandas as pd

file_name = 'cer_wer_output.csv'

df = pd.read_csv(file_name)

df.head()

,audio_name,human_text,human_text_norm,transcript_original,transcript_synthesized,cer_original,cer_synthesized,wer_original,wer_synthesized
0,sr16k_mono_0027-CP011_1250.69-1261.3,"Ô Jorge, é o seguinte, a gente esqueceu de avi...",ô jorge é o seguinte a gente esqueceu de avisa...,o jorge é o seguinte a gente esqueceu de avisa...,eu já é o seguinte a gente esqueceu de avisar ...,0.037267,0.211180,0.161290,0.387097
1,sr16k_mono_0038-CP015_950.037-979.634,E eu gostaria de lembrar que um dos motivos da...,e eu gostaria de lembrar que um dos motivos da...,eu gostaria de lembrar que um dos motivos dess...,eu gostaria de lembrar que os motivos da da re...,0.104623,0.085158,0.184211,0.118421
2,sr16k_mono_0067-CP546_1576.56-1591.05,"É. é que Veja bem, essa experiência, eu gostar...",é é que veja bem essa experiência eu gostaria ...,é eh que veja bem essa experiência eu gostaria...,é que veja bem essa experiência eu gostaria qu...,0.102041,0.073469,0.170213,0.212766
3,sr16k_mono_0082-CP707_1508.09-1518.64,"Mas eu gostaria... Claudio, dá dá só uma uma u...",mas eu gostaria claudio dá dá só uma uma uma a...,mas eu gostaria caldo dá dá só uma uma aperfei...,você gostaria de é de dada só uma uma aperfeiç...,0.112583,0.172185,0.214286,0.321429
4,sr16k_mono_0079-CP623_853.479-861.621,Foi nesse contraponto que eu tentei compreende...,foi nesse contraponto que eu tentei compreende...,então foi nesse contraponto que eu tentei comp...,foi nesse contrário conto que eu tentei compre...,0.148148,0.194444,0.263158,0.421053


In [ ]:
import os
import csv
import pandas as pd


df = pd.read_csv('cer_wer_output.csv')

file_name = 'cer_wer_output_v2.csv'

folder_path = 'resulting_audios_cv-fn'

with open(file_name, 'a', newline='') as csv_file:
    writer = csv.writer(csv_file)

    csv_data = ["audio_name", "human_text", "human_text_norm", "transcript_original", "transcript_synthesized_yourtts", "transcript_synthesized_f5tts_cv", "cer_original", "cer_synthesized_yourtts", "cer_synthesized_f5tts_cv", "wer_original", "wer_synthesized_yourtts", "wer_synthesized_f5tts_cv"]
    writer.writerow(csv_data)

    for filename in os.listdir(folder_path):
        if filename.endswith(".wav"):
            audio_name = filename[:-4]
            print(audio_name)

            audio_key = f"sr16k_mono_{audio_name[6:]}"
            print(audio_key)

            row = df.query(f"audio_name == '{audio_key}'")
            print(row['audio_name'].iloc[0])
            print(row['human_text_norm'].iloc[0])
            print("-------")
            
            text_norm = replace_special_tokens_and_normalize(row['human_text_norm'].iloc[0])

            transcript_f5cv = trancript_with_distil(f"{folder_path}/{filename}")
            transcript_f5cv_norm = replace_special_tokens_and_normalize(transcript_f5cv)

            f5cv_wer_cer = calculate_wer_cer(text_norm, transcript_f5cv_norm)

            print("-", text_norm)
            print("-", transcript_f5cv_norm)
            print(f5cv_wer_cer)

            csv_data = [audio_key, row["human_text"].iloc[0], row["human_text_norm"].iloc[0], row["transcript_original"].iloc[0], row["transcript_synthesized"].iloc[0], transcript_f5cv_norm, row["cer_original"].iloc[0], row["cer_synthesized"].iloc[0], f5cv_wer_cer[1], row["wer_original"].iloc[0], row["wer_synthesized"].iloc[0], f5cv_wer_cer[0]]
            writer.writerow(csv_data)

            #break


In [23]:
from scipy import stats
import pandas as pd

file_name = 'cer_wer_output_v2.csv'

df = pd.read_csv(file_name)

t_statistic, p_value = stats.ttest_ind(df["wer_original"], df["wer_synthesized_f5tts_cv"])

print(f"WER ORIGINAL: {df["wer_original"].mean()*100:.2f}")
print(f"WER SINTETIZADO F5 CV: {df["wer_synthesized_f5tts_cv"].mean()*100:.2f}")

print(f"T-statistic: {t_statistic:.4f} ({t_statistic})")
print(f"P-value: {p_value:.4f} ({p_value})" )

WER ORIGINAL: 13.60
WER SINTETIZADO F5 CV: 18.12
T-statistic: -3.0251 (-3.025088762473149)
P-value: 0.0028 (0.0028187078083642975)


In [24]:
from scipy import stats
import pandas as pd

file_name = 'cer_wer_output_v2.csv'

df = pd.read_csv(file_name)

t_statistic, p_value = stats.ttest_ind(df["wer_synthesized_yourtts"], df["wer_synthesized_f5tts_cv"])

print(f"WER ORIGINAL: {df["wer_synthesized_yourtts"].mean()*100:.2f}")
print(f"WER SINTETIZADO F5 CV: {df["wer_synthesized_f5tts_cv"].mean()*100:.2f}")

print(f"T-statistic: {t_statistic:.4f} ({t_statistic})")
print(f"P-value: {p_value:.4f} ({p_value})" )

WER ORIGINAL: 23.77
WER SINTETIZADO F5 CV: 18.12
T-statistic: 3.6120 (3.6120189458795777)
P-value: 0.0004 (0.0003859933928752929)
